In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten
from keras import regularizers
from keras.optimizers import SGD
from keras import losses
from keras import utils
from keras import callbacks

Using TensorFlow backend.


In [2]:
import numpy as np
from PIL import Image
import copy

# constants
CLASS = 200
SAMPLES_PER_CLASS = 500
TOTAL_SAMPLES = CLASS * SAMPLES_PER_CLASS
COLOR_CHANNELS = 3
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
VAL_SAMPLES = 10000

# read all of the word net id
wnids = [id.strip('\n') for id in open('/datasets/tmp/cg181fdn/tiny-imagenet-200/wnids.txt').readlines()]

# data will store all of the data
data = {}

# train data
data['train'] = {}
data['train']['data'] = np.ndarray(shape=(TOTAL_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, COLOR_CHANNELS), dtype=np.uint8)
data['train']['target'] = np.ndarray(shape=(TOTAL_SAMPLES,), dtype=np.uint8)

# validation data
data['val'] = {}
data['val']['data'] = np.ndarray(shape=(VAL_SAMPLES, IMAGE_WIDTH, IMAGE_HEIGHT, COLOR_CHANNELS), dtype=np.uint8)
data['val']['target'] = np.ndarray(shape=(VAL_SAMPLES,), dtype=np.uint8)

# iterate through work net ids
print("storing training:")
for i in range(len(wnids)):
    wnid = wnids[i]
    print("%s: %d / %d" % (wnid, i + 1, len(wnids)))
    for j in range(500):
        temp = []
        path = "/datasets/tmp/cg181fdn/tiny-imagenet-200/train/{0}/images/{0}_{1}.JPEG".format(wnid, j)
        test = np.array(Image.open(path).convert('RGB'))
        data['train']['data'][i * SAMPLES_PER_CLASS + j] = test
        data['train']['target'][i * SAMPLES_PER_CLASS + j] = i

# get the validation data
print("storing validation:")
for i, line in enumerate(map(lambda s: s.strip(), open('/datasets/tmp/cg181fdn/tiny-imagenet-200/val/val_annotations.txt'))):
    name, wnid = line.split('\t')[0:2]
    temp = []
    path = "/datasets/tmp/cg181fdn/tiny-imagenet-200/val/images/{0}".format(name)
    test = np.array(Image.open(path).convert('RGB'))
    data['val']['data'][i] = test
    data['val']['target'][i] = wnids.index(wnid)

storing training:
n02124075: 1 / 200
n04067472: 2 / 200
n04540053: 3 / 200
n04099969: 4 / 200
n07749582: 5 / 200
n01641577: 6 / 200
n02802426: 7 / 200
n09246464: 8 / 200
n07920052: 9 / 200
n03970156: 10 / 200
n03891332: 11 / 200
n02106662: 12 / 200
n03201208: 13 / 200
n02279972: 14 / 200
n02132136: 15 / 200
n04146614: 16 / 200
n07873807: 17 / 200
n02364673: 18 / 200
n04507155: 19 / 200
n03854065: 20 / 200
n03838899: 21 / 200
n03733131: 22 / 200
n01443537: 23 / 200
n07875152: 24 / 200
n03544143: 25 / 200
n09428293: 26 / 200
n03085013: 27 / 200
n02437312: 28 / 200
n07614500: 29 / 200
n03804744: 30 / 200
n04265275: 31 / 200
n02963159: 32 / 200
n02486410: 33 / 200
n01944390: 34 / 200
n09256479: 35 / 200
n02058221: 36 / 200
n04275548: 37 / 200
n02321529: 38 / 200
n02769748: 39 / 200
n02099712: 40 / 200
n07695742: 41 / 200
n02056570: 42 / 200
n02281406: 43 / 200
n01774750: 44 / 200
n02509815: 45 / 200
n03983396: 46 / 200
n07753592: 47 / 200
n04254777: 48 / 200
n02233338: 49 / 200
n04008634: 

In [3]:
shuffle_train_index = np.random.permutation(len(data['train']['data']))
training_images = np.array(data['train']['data'])[shuffle_train_index]
training_labels = np.array(data['train']['target'])[shuffle_train_index]
training_labels = utils.to_categorical(training_labels, num_classes=200)

val_images = np.array(data['val']['data'])
val_labels = np.array(data['val']['target'])
val_labels = utils.to_categorical(val_labels, num_classes=200)

In [4]:
class TestCallback(callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [7]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='softmax'))

sgd = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,
             metrics=['accuracy'])

hist = model.fit(x=training_images, y=training_labels, batch_size=128,
                 epochs=30, callbacks=[TestCallback((val_images, val_labels))])
#score,acc = model.evaluate(val_images, val_labels, batch_size=32)

Epoch 1/30
 99968/100000 [============================>.] - ETA: 0s - loss: 12.2027 - acc: 0.0055
Testing loss: 5.30729685822, acc: 0.0047

100000/100000 [==============================] - 25s - loss: 12.2006 - acc: 0.0055    
Epoch 2/30
 99968/100000 [============================>.] - ETA: 0s - loss: 5.3719 - acc: 0.0050
Testing loss: 5.29912235718, acc: 0.0048

100000/100000 [==============================] - 25s - loss: 5.3719 - acc: 0.0050    
Epoch 3/30
 99968/100000 [============================>.] - ETA: 0s - loss: 5.3253 - acc: 0.0046
Testing loss: 5.29838058319, acc: 0.0036

100000/100000 [==============================] - 26s - loss: 5.3252 - acc: 0.0046    
Epoch 4/30
 99968/100000 [============================>.] - ETA: 0s - loss: 5.3136 - acc: 0.0050
Testing loss: 5.2983504425, acc: 0.004

100000/100000 [==============================] - 25s - loss: 5.3136 - acc: 0.0050    
Epoch 5/30
 99968/100000 [============================>.] - ETA: 0s - loss: 5.3085 - acc: 0.0050
Tes